In [2]:
import tablescraper as scrape
import requests
import re
import pandas as pd

In [3]:
# bdays = scrape.scrape_elems(
#     ['https://en.wikipedia.org/wiki/Robert_Downey_Jr.', 'https://en.wikipedia.org/wiki/Paul_Bettany'],
#     '//*[@class="bday"]/text()')

# bdays

['1965-04-04', '1971-05-27']

## Get wikipedia page by actor_name, then get birthday

In [75]:
def wikipedia_urls(actor_names):
    """
    Returns wikipedia search urls, formatted as a dict like {actor_name:wiki_page_url, ...}
    Warning: will return a 'disambiguation page' on some searches
    """

    actor_names = scrape.to_sequence(actor_names)
    final_urls = {}
    
    for actor_name in actor_names:

        url = 'https://en.wikipedia.org/w/index.php?search={name}&title=Special%3ASearch&go=Go'.format(
            name=re.sub(r"[^A-Za-z+]+", '', actor_name.lower().replace(' ','+')))

        final_urls[actor_name] = url

    return final_urls

actor_revenue = pd.read_pickle('actor_revenue.pickle')
actor_revenue.head()

,rank,title,studio,adjusted_gross,unadjusted_gross,release,actor_id,actor_name
0,-,Marvel's The Avengers,BV,668866600,623357910,2012-05-04,paulbettany,Paul Bettany
1,1,Avengers: Age of Ultron,BV,465684200,459005868,2015-05-01,paulbettany,Paul Bettany
2,-,Iron Man 3,BV,424632700,409013994,2013-05-03,paulbettany,Paul Bettany
3,-,Iron Man,Par.,385808100,318412101,2008-05-02,paulbettany,Paul Bettany
4,-,Iron Man 2,Par.,341908200,312433331,2010-05-07,paulbettany,Paul Bettany


In [4]:
# actor_names = actor_revenue.actor_name.unique()

# wiki_urls = wikipedia_urls(actor_names)
# birthdays = scrape.scrape_elems(wiki_urls, '//*[@class="bday"]/text()')

# print len(birthdays)
# print zip(birthdays, actor_names)[:5]

In [5]:
actor_names = actor_revenue.actor_name.unique()

print len(actor_names)

wiki_urls = wikipedia_urls(actor_names)
wiki_data = scrape.scrape_multi_elems(wiki_urls, 
    {
        'bday':'//*[@class="bday"]/text()',
        'text':'normalize-space(string(//div[@id="mw-content-text"]))',
        'categories':'(//*[@id="catlinks"]//a)[position()>1]/text()'
    },
    keep_trees=True)

print len(wiki_data)

781
781


In [7]:
# pickle can't handle fancy dictionaries, dill to the rescue
import dill

# with open('wiki_data_dict.dill', 'wb') as f:
#     dill.dump(wiki_data, f)
    
# !ls

actor-demographics.ipynb	       scrape_boxofficemojo_sandbox.ipynb
actor-grosses-mojo.ipynb	       scraping_actor_info__the_numbers.ipynb
actor_revenue.pickle		       tablescraper.log
boxofficemojo.pyc		       tablescraper.py
data				       tablescraper.pyc
README.md			       wiki_data_dict.dill
render_html_in_ipython_notebook.ipynb  wiki_data_dict.pickle


In [8]:
test_name = actor_names[0]

print test_name

print wiki_data[test_name]['bday']
print wiki_data[test_name]['text'][:100]
print wiki_data[test_name]['categories']
print wiki_data[test_name]['tree']

print type(wiki_data[test_name]['tree'])

Paul Bettany
['1971-05-27']
An automated process has detected links on this page on the local or global blacklist. If the links 
['1971 births', 'Male actors from London', 'Alumni of the Drama Centre London', 'British buskers', 'English male film actors', 'English male stage actors', 'British atheists', 'British expatriates in the United States', 'Living people', 'People from Harlesden', 'Royal Shakespeare Company members', '20th-century British male actors', '21st-century British male actors', "People from Shepherd's Bush", 'Tagged pages containing blacklisted links', 'Articles with hCards', 'All articles with unsourced statements', 'Articles with unsourced statements from March 2014', 'Commons category with page title same as on Wikidata', 'Use British English from August 2010', 'Use dmy dates from June 2011', 'Wikipedia articles with VIAF identifiers', 'Wikipedia articles with LCCN identifiers', 'Wikipedia articles with ISNI identifiers', 'Wikipedia articles with GND identifiers', '

I want to use the Wikipedia categories as features -- they'll be one-hot columns aka dummy variables. 

So first I need to figure out all the unique categories, then for each actor change `categories` to a pandas Series indexed by the category name, with True for 'has category' and False for 'does not have category'.

In [9]:
all_categories = set()

for actor_name, actor_data in wiki_data.iteritems():
    all_categories.update(actor_data['categories'])

all_categories = list(all_categories)
print 'got %i unique categories\n' % len(all_categories)
print '\n'.join(all_categories[:10])

got 4826 unique categories

Deaths from fire in the United States
Detroit Country Day School alumni
21st-century Australian actresses
Kwantlen Polytechnic University alumni
Female comics writers
Articles containing potentially dated statements from March 2014
Articles containing potentially dated statements from March 2016
Use Australian English from February 2014
O. Henry Award winners
Scottish nationalists


In [10]:
for actor_name, actor_data in wiki_data.iteritems():
    for category in all_categories:
        wiki_data[actor_name][category] = category in actor_data['categories']

In [11]:
wiki_df = pd.DataFrame.from_dict(wiki_data, orient='index').drop(['tree','categories','text'], axis=1)
wiki_df.head()

,Detroit Country Day School alumni,American punk rock musicians,Kwantlen Polytechnic University alumni,Female comics writers,Articles containing potentially dated statements from March 2014,Articles containing potentially dated statements from March 2016,Use Australian English from February 2014,O. Henry Award winners,Scottish nationalists,"People from Wyckoff, New Jersey",...,American sopranos,"People from Ashland, Kentucky",Trip hop musicians,Tibetan Buddhists from Singapore,Male actors from Guangzhou,"Male actors from Evanston, Illinois","People from Westchester County, New York",American gun control advocates,Kickboxers from Tennessee,"People educated at Wellington College, Berkshire"
50 Cent,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Aaron Eckhart,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Aaron Johnson,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Abbie Cornish,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Abigail Breslin,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
# Flatten the birthday 1-lists, then convert to Timestamp
wiki_df.bday = pd.to_datetime([bday[0] if len(bday)==1 else '' for bday in wiki_df.bday])
wiki_df.bday.head()

50 Cent                  NaT
Aaron Eckhart     1968-03-12
Aaron Johnson     1990-06-13
Abbie Cornish     1982-08-07
Abigail Breslin   1996-04-14
Name: bday, dtype: datetime64[ns]

I don't necessarily have to do this, but I'll remove the features that are too sparse by setting a threshold. If less than `category_cutoff` people have feature X, that feature will be removed.

In [63]:
category_cutoff = 10

summed_categories = wiki_df.sum()
unimportant_categories = summed_categories[summed_categories < category_cutoff]

print '%i unimportant categories to drop (less than than %i members)' % (len(unimportant_categories), category_cutoff)
unimportant_categories.head()

4487 unimportant categories to drop (less than than 10 members)


Detroit Country Day School alumni                                   1
American punk rock musicians                                        1
Kwantlen Polytechnic University alumni                              1
Female comics writers                                               2
Articles containing potentially dated statements from March 2014    1
dtype: int64

In [64]:
# filter out the unimportand categories

filtered_wiki_df = wiki_df.drop(unimportant_categories.index, axis=1)
filtered_wiki_df.head()

,Canadian male voice actors,21st-century Australian actresses,Canadian male television actors,English people of Irish descent,Commanders of the Order of the British Empire,Wikipedia protected pages without expiry,1992 births,English male television actors,Best Supporting Actress Academy Award winners,Articles with unsourced statements from January 2016,...,Australian people of English descent,Articles with DMOZ links,Australian television actresses,Articles with dead external links from October 2010,Best Miniseries or Television Movie Actor Golden Globe winners,British Shakespearean actresses,American television directors,American male child actors,Articles with unsourced statements from February 2016,Articles with unsourced statements from April 2016
50 Cent,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Aaron Eckhart,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Aaron Johnson,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
Abbie Cornish,False,True,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
Abigail Breslin,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [65]:
# bday still OK?
filtered_wiki_df.bday.head()

50 Cent                  NaT
Aaron Eckhart     1968-03-12
Aaron Johnson     1990-06-13
Abbie Cornish     1982-08-07
Abigail Breslin   1996-04-14
Name: bday, dtype: datetime64[ns]

Now I want to **join** actor biographic info from `filtered_wiki_df` to actor movie info from `actor_revenue`

In [76]:
actor_full = actor_revenue.set_index('actor_name').join(filtered_wiki_df)
actor_full.head()

,rank,title,studio,adjusted_gross,unadjusted_gross,release,actor_id,Canadian male voice actors,21st-century Australian actresses,Canadian male television actors,...,Australian people of English descent,Articles with DMOZ links,Australian television actresses,Articles with dead external links from October 2010,Best Miniseries or Television Movie Actor Golden Globe winners,British Shakespearean actresses,American television directors,American male child actors,Articles with unsourced statements from February 2016,Articles with unsourced statements from April 2016
50 Cent,1,Spy,Fox,116867700,110825712,2015-06-05,50cent,False,False,False,...,False,False,False,False,False,False,False,False,False,False
50 Cent,2,Southpaw,Wein.,55265500,52421953,2015-07-24,50cent,False,False,False,...,False,False,False,False,False,False,False,False,False,False
50 Cent,3,Righteous Kill,Over.,48566600,40081410,2008-09-12,50cent,False,False,False,...,False,False,False,False,False,False,False,False,False,False
50 Cent,4,Get Rich or Die Tryin',Par.,42048300,30985352,2005-11-09,50cent,False,False,False,...,False,False,False,False,False,False,False,False,False,False
50 Cent,5,Escape Plan,LG/S,26202200,25135965,2013-10-18,50cent,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [78]:
# finally add a column for actor age on release date
actor_full['age_on_release'] = actor_full.release - actor_full.bday

actor_full[['age_on_release','bday','release','title','adjusted_gross']].tail()

,age_on_release,bday,release,title,adjusted_gross
Zooey Deschanel,9038 days,1980-01-17,2004-10-15,Eulogy,105200
Zooey Deschanel,8499 days,1980-01-17,2003-04-25,Manic,100900
Zooey Deschanel,9934 days,1980-01-17,2007-03-30,Live Free or Die,17500
Zooey Deschanel,10368 days,1980-01-17,2008-06-06,The Go-Getter,14500
Zooey Deschanel,10198 days,1980-01-17,2007-12-19,Flakes,1000


In [89]:
print '%i rows, %i columns' % actor_full.shape

16416 rows, 348 columns


Enable to pickle:

In [79]:
# actor_full.to_pickle('actor_full.pickle')

# The stuff under this uses old vars...

In [12]:
bday_series = pd.Series(data=pd.to_datetime(birthdays), index=actor_names, name='birthday')
print 'Got %i birthdays out of %i' % (len(bday_series[~bday_series.isnull()]), len(actor_names))
bday_series.head()

Got 729 birthdays out of 781


Paul Bettany       1971-05-27
Kate Bosworth      1983-01-02
Christine Taylor   1971-07-30
Kathy Bates        1948-06-28
Jack Davenport     1973-03-01
Name: birthday, dtype: datetime64[ns]

In [7]:
actor_rev_bday = actor_revenue.set_index('actor_name').join(bday_series)
actor_rev_bday.tail()

,rank,title,studio,adjusted_gross,unadjusted_gross,release,actor_id,birthday
Zooey Deschanel,21,Eulogy,Lions,"$105,200","$75,076",10/15/04,zooeydeschanel,1980-01-17
Zooey Deschanel,22,Manic,IFC,"$100,900","$69,958",4/25/03,zooeydeschanel,1980-01-17
Zooey Deschanel,23,Live Free or Die,Think,"$17,500","$13,836",3/30/07,zooeydeschanel,1980-01-17
Zooey Deschanel,24,The Go-Getter,PArch,"$14,500","$11,931",6/6/08,zooeydeschanel,1980-01-17
Zooey Deschanel,25,Flakes,IFC,"$1,000",$778,12/19/07,zooeydeschanel,1980-01-17


Which birthdays did we miss?

In [14]:
actor_rev_bday[actor_rev_bday.birthday.isnull()].index.unique()

array(['50 Cent', 'Adam Scott', 'Albert Brooks', 'Alex Kendrick',
       'Alex Russell', 'Ben Barnes', 'Billy Burke', 'Bow Wow',
       'Boyd Holbrook', 'Brian Cox', 'Carrie-Anne Moss',
       'Catherine Zeta-Jones', "Chris 'Ludacris' Bridges", 'Chris Evans',
       'Chris Klein', "Chris O'Donnell", 'Christopher Mintz-Plasse',
       'Common', 'Daniel Day-Lewis', 'Dustin Ingram', 'Elizabeth Debicki',
       'Ellen Wong', 'Famke Janssen', 'Ice Cube', "Jack O'Connell",
       'Jada Pinkett-Smith', "Jason O'Mara", 'Jeffrey Wright',
       'Jenna Dewan-Tatum', 'Joseph Gordon-Levitt', 'Joseph Schneider',
       'Josh Zuckerman', 'Kenny Wormald', 'Kodi Smit-McPhee',
       'Macon Blair', 'Madonna', 'Mary-Kate Olsen', 'Mia Wasikowska',
       'Michelle Williams', "Mo'Nique", 'Naomie Harris', 'Ne-Yo',
       'Patrick Wilson', 'Prabhu Dheva', 'Ray Stevenson',
       'Rosie Huntington-Whiteley', "Sean 'P. Diddy' Combs",
       u'Sebasti\xe1n Silva', 'Thomas Dekker', 'Tom Arnold', 'Tom Holland',


It turns out these names will land you in a wikipedia disambiguation page, eg https://en.wikipedia.org/wiki/Adam_Scott

I *could* re-scrape these names and follow the link that looks like `Adam Scott (actor)` to find the actor's page... but I'll ignore them for now.

# Gender? 

You could calculate the "her"+"she" to "his"+"he" ratio in the Wikipedia page text...

Or you could search the categories for 'male'/'female' ratio.

Or, the existing categories might be enough!